[Apache Parquet](https://arrow.apache.org/docs/python/parquet.html) is an efficient columnar storage format. Compared to saving this dataset in csvs using parquet:
- Greatly reduces the necessary disk space
- Loads the data into Pandas with memory efficient datatypes
- Enables fast reads from disk
- Allows us to easily work with partitions of the data

Pandas has a parquet integration that makes loading data into a dataframe trivial; we'll try that now.

In [1]:
import pandas as pd

In [2]:
book_train = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '../input/optiver-realized-volatility-prediction/book_train.parquet'

If this data were stored as a csv, the numeric types would all default to the 64 bit versions. Parquet retains the more efficient types I specified while saving the data.

**Expect memory usage to spike to roughly double the final dataframe size while parquet loads a file. Consider loading your largest dataset first or using partitions to mitigate this.**

In [ ]:
book_train.info()

The one exception is the `stock_id` column, which has been converted to the category type as it is [the partition column](https://arrow.apache.org/docs/python/parquet.html#reading-from-partitioned-datasets). The parquet files in this dataset are all paritioned by `stock_id` so that it's not necessary to load the entire file at once. In fact, if you examine the parquet files you'll see that they are actually directories.

In [ ]:
! ls ../input/optiver-realized-volatility-prediction/book_train.parquet | head -n 5

Those are in turn also directories, which would be relevant if the data were partitioned by more than one column.

In [ ]:
! ls ../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/

In [ ]:
book_train_0 = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet')
book_train_0.info()

Note that because we loaded a single partition, **the partition column was not included**. We could remedy that manually if we need the stock ID or just load a larger subset of the data by passing a list of paths. This will load all of the stock IDs 110-119, reducing memory usesage without implicitly dropping the partition column:

In [ ]:
import glob
subset_paths = glob.glob('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=11*/*')
book_train_subset = pd.read_parquet(subset_paths)
book_train_subset.info()

In [9]:
import numpy as np
list = [True,2,'j','4']
np.array(list, dtype=object)

array([True, 2, 'j', '4'], dtype=object)